# BAX pore assembly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

%matplotlib inline

In [ ]:
slice_vec = np.array([12, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0])

## Load data

### Tree tensor integrator

#### Partition 0
(0 1 2)(((3 4 6 7)(5 8))(9 10))

In [ ]:
tree = readTree("output/bax_p0_r5-10-20_i_tau5e-2/output_t2900.nc")
P_slice_tensor0, P_sum_tensor0 = tree.calculateObservables(0, slice_vec)

#### Partition 1
((0 1 2)(3 4 5))((6 7 9)(8 10))

In [ ]:
tree = readTree("output/bax_p1_r5-15-15_i_tau5e-2/output_t2900.nc")
P_slice_tensor1, P_sum_tensor1 = tree.calculateObservables(0, slice_vec)

#### Partition 2
((0 1)(2 3 4))((5 6 7 8)(9 10))

In [ ]:
tree = readTree("output/bax_p2_r5-25-25_i_tau5e-2/output_t2900.nc")
P_slice_tensor2, P_sum_tensor2 = tree.calculateObservables(0, slice_vec)

#### Two partitions
(0 1 2 3 4)(5 6 7 8 9 10)

In [ ]:
tree = readTree("output/bax_pfull_r5_i_tau5e-2/output_t2900.nc")
P_slice_tensor_full, P_sum_tensor_full = tree.calculateObservables(0, slice_vec)

### Matrix integrator

In [ ]:
with xr.open_dataset("output/bax_general_230730_r5_so_tau1e0_ss100/output_t100.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    idx_2D = np.array([0, 1])
    P_sum_matrix = lr_sol.marginalDistributions()
    P_slice_matrix = lr_sol.slicedDistributions(slice_vec)

### SSA

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e6.npy")
ssa_sol = SSASol(result)
P_sum_ssa, _, P_slice_ssa, _ = ssa_sol.calculateObservables(slice_vec, idx_2D)

## Comparison of the sliced distributions

### Logarithmic plot

In [ ]:
plt.figure(1)
plt.plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], 'o', fillstyle="none", label="DLRA")
plt.plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor1[11:26], 'x:', fillstyle="none", label="hDLRA ($P_1$)")
plt.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 's', fillstyle="none", label="SSA ($10^7$ runs)")
plt.legend()
plt.yscale("log")
plt.xlabel("$x_0$")
plt.ylabel("$P_\\mathrm{{S}}(x_0)$")
plt.xlim([10, 26]);

### Ordinary plot

In [ ]:
plt.figure(2)
plt.plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], 'o', fillstyle="none", label="DLRA")
plt.plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor1[11:26], 'x:', fillstyle="none", label="hDLRA ($P_1$)")
plt.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 's', fillstyle="none", label="SSA ($10^7$ runs)")
plt.legend()
plt.xlabel("$x_0$")
plt.ylabel("$P_\\mathrm{{S}}(x_0)$")
plt.xlim([10, 26]);

In [ ]:
plt.figure(3)
plt.plot(np.arange(grid.n[0]), P_sum_matrix[0], 'o', fillstyle="none", label="DLRA")
plt.plot(np.arange(tree.root.child[0].grid.n[0]), P_sum_tensor1, 'x:', fillstyle="none", label="hDLRA (P_1)")
plt.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0]), P_sum_ssa[-1][0], 's', fillstyle="none", label="SSA ($10^7$ runs)")
plt.legend()
plt.xlabel("$x_0$")
plt.ylabel("$P_\\mathrm{{M}}(x_0)$");

## Comparison with deterministic solution

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
wall_time = time_series.getWallTime()
t, concentrations = time_series.calculateConcentrations()

In [ ]:
from pysb.integrate import odesolve

from scripts.models.bax_pysb import model

In [ ]:
concentrations_ode = odesolve(model, t)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(t, concentrations, '-')
ax1.set_title("CME")
ax1.set_ylabel("$c_i(t)$")

for o in model.observables:
    ax2.plot(t, concentrations_ode[o.name], '-', label=o.name[1:])
ax2.set_title("ODE")
ax2.legend()
plt.setp((ax1, ax2), ylim=[0.0, 52.0], xlabel="$t$");

In [ ]:
plt.figure(5)
for i, o in enumerate(model.observables):
    plt.plot(t, np.abs((concentrations[:, i]-concentrations_ode[o.name])), '-')
plt.xlabel("$t$")
plt.ylabel("$|c_i^\mathrm{{CME}}(t) - c_i^\mathrm{{ODE}}(t)|$");

In [ ]:
time_series = TimeSeries("output/bax_p0_r5-10-20_i_tau5e-2")
t_p0, concentrations_p0 = time_series.calculateConcentrations()

In [ ]:
time_series = TimeSeries("output/bax_p2_r5-25-25_i_tau5e-2")
t_p2, concentrations_p2 = time_series.calculateConcentrations()

In [ ]:
time_series = TimeSeries("output/bax_pfull_r5_i_tau5e-2")
t_full, concentrations_full = time_series.calculateConcentrations()

In [ ]:
concentrations_ssa = np.zeros(ssa_sol.n_time)
for i in range(ssa_sol.n_time):
    concentrations_ssa[i] = np.dot(P_sum_ssa[i][-2], np.arange(ssa_sol.n[-2]) + ssa_sol.n_min[-2])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(t_p0, concentrations_p0[:, -2], '-', label="hDLRA ($P_0$)")
ax1.plot(t, concentrations[:, -2], '-', label="hDLRA ($P_1$)")
ax1.plot(t_p2, concentrations_p2[:, -2], '-', label="hDLRA ($P_2$)")
ax1.plot(t_full, concentrations_full[:, -2], 'x', label="hDLRA (2 partitions)")
ax1.plot([0.0, 145.0], concentrations_ssa, 's', label="SSA ($10^7$ runs)")
ax1.set_title("CME")
ax1.legend()

ax2.plot(t, concentrations_ode["omSmac"], '-')
ax2.set_title("ODE")
plt.setp((ax1, ax2), ylim=[49.6, 50.6], xlabel="$t$");